In [1]:
!pip install faker

In [2]:
 !pip install pymysql

In [3]:
import random
import pymysql
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()

In [ ]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
num_person = 100

# Generate PERSON records
print("-- INSERT INTO PERSON Table")

for person_id in range(1, num_person + 1):
    fname = fake.first_name()
    lname = fake.last_name()
    age = random.randint(18, 64)
    gender = random.choice(['M', 'F'])
    addr1 = fake.street_address()
    addr2 = fake.secondary_address()
    city = fake.city()
    state = fake.state_abbr()
    zipcode = fake.zipcode()

    sql = """
        INSERT INTO Person (PersonID, FirstName, LastName, Age, Gender, AddressLine1, AddressLine2, City, State, ZipCode)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (person_id, fname, lname, age, gender, addr1, addr2, city, state, zipcode)
    cursor.execute(sql, values)

conn.commit()
print("✅ Person records inserted.")
cursor.close()
conn.close()

-- INSERT INTO PERSON Table


IntegrityError: (1062, "Duplicate entry '1' for key 'person.PRIMARY'")

In [16]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

NUM_DEPARTMENTS = 5

# Generate DEPARTMENT records
print("-- INSERT INTO DEPARTMENT Table")

# Insert Departments
for dept_id in range(1, NUM_DEPARTMENTS + 1):
    cursor.execute("INSERT INTO Department (DepartmentID, DepartmentName) VALUES (%s, %s)",
                   (dept_id, fake.bs().title()))

# Commit and close connection
conn.commit()
print("✅ Fake data inserted into Department table successfully!")

cursor.close()
conn.close()


-- INSERT INTO DEPARTMENT Table


IntegrityError: (1062, "Duplicate entry '1' for key 'department.PRIMARY'")

In [ ]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

num_phones = 100  # assuming 1 phone per person
print("-- INSERT INTO PHONE Table")

for phone_id in range(1, num_phones + 1):
    person_id = phone_id  # assuming one phone per person
    phone_number = fake.phone_number()[:20]

    sql = """
        INSERT INTO Phone (PhoneID, PersonID, PhoneNumber)
        VALUES (%s, %s, %s)
    """
    values = (phone_id, person_id, phone_number)
    cursor.execute(sql, values)

conn.commit()
print("✅ Phone records inserted.")
cursor.close()
conn.close()

-- INSERT INTO Phone Table
✅ Phone records inserted.


In [3]:
import pymysql
from faker import Faker
import re

fake = Faker()

# Connect to MySQL
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

# Regex pattern is not needed here because we format numbers ourselves,
# but keeping a function to generate valid phone numbers:

def get_valid_phone():
    """Generate a valid phone number in the format xxx.xxx.xxxx"""
    while True:
        phone = fake.phone_number()
        # Remove all non-digit characters
        digits = re.sub(r'\D', '', phone)
        if len(digits) == 10:
            return f"{digits[:3]}.{digits[3:6]}.{digits[6:]}"

# Get current max PhoneID to avoid duplicates
cursor.execute("SELECT MAX(PhoneID) FROM Phone")
max_id = cursor.fetchone()[0] or 0
next_id = max_id + 1

# Fetch all PersonIDs (assuming you want to assign one phone per person)
cursor.execute("SELECT PersonID FROM Person")
person_ids = [row[0] for row in cursor.fetchall()]

print("-- INSERTING cleaned phone numbers")

for i, person_id in enumerate(person_ids):
    phone_id = next_id + i
    phone_number = get_valid_phone()

    sql = """
        INSERT INTO Phone (PhoneID, PersonID, PhoneNumber)
        VALUES (%s, %s, %s)
    """
    try:
        cursor.execute(sql, (phone_id, person_id, phone_number))
    except pymysql.err.IntegrityError as e:
        print(f"❌ Skipping PhoneID {phone_id} for PersonID {person_id}: {e}")

conn.commit()
print("✅ Clean phone numbers inserted.")
cursor.close()
conn.close()


-- INSERTING cleaned phone numbers
✅ Clean phone numbers inserted.


In [40]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

# Step 1: Fetch existing PersonIDs and DepartmentIDs
cursor.execute("SELECT PersonID FROM Person")
person_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT DepartmentID FROM Department")
department_ids = [row[0] for row in cursor.fetchall()]

# Step 2: Generate Employees using existing PersonIDs
print("-- INSERT INTO Employee")

num_employees = min(20, len(person_ids))  # Ensure we don’t exceed available people
used_person_ids = set()

for _ in range(num_employees):
    person_id = random.choice(person_ids)
    if person_id in used_person_ids:
        continue  # Skip if this person is already an employee
    used_person_ids.add(person_id)

    title = random.choice(['Manager', 'Engineer', 'Sales Rep', 'Clerk', 'Technician'])
    erank = random.randint(1, 5)

    # SupervisorID is omitted from insertion
    sql = """
        INSERT INTO Employee (PersonID, Title, Erank)
        VALUES (%s, %s, %s)
    """
    values = (person_id, title, erank)
    cursor.execute(sql, values)

conn.commit()
print("✅ Employee records inserted without SupervisorID.")

cursor.close()
conn.close()

-- INSERT INTO Employee


IntegrityError: (1062, "Duplicate entry '77' for key 'employee.PRIMARY'")

In [9]:
import pymysql
import random

# Define a more diverse set of realistic titles
titles = [
    'Software Engineer', 'Senior Engineer', 'Project Manager', 'Technical Lead',
    'Data Analyst', 'Systems Administrator', 'Network Engineer', 'DevOps Engineer',
    'Business Analyst', 'HR Specialist', 'Marketing Manager', 'Product Designer',
    'Support Specialist', 'Finance Officer', 'Security Analyst', 'QA Tester'
]

# Connect to your MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz',
    autocommit=True
)

cursor = conn.cursor()

# Fetch existing employee PersonIDs
cursor.execute("SELECT PersonID FROM Employee")
employee_ids = [row[0] for row in cursor.fetchall()]

# Assign new titles
for person_id in employee_ids:
    new_title = random.choice(titles)
    cursor.execute(
        "UPDATE Employee SET Title = %s WHERE PersonID = %s",
        (new_title, person_id)
    )

print("✅ Employee titles updated successfully.")

cursor.close()
conn.close()


✅ Employee titles updated successfully.


In [32]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- UPDATE Employee SET SupervisorID")

# Step 1: Fetch all PersonIDs from Employee
cursor.execute("SELECT PersonID FROM Employee")
all_employees = [row[0] for row in cursor.fetchall()]

for emp_id in all_employees:
    # Choose a supervisor who is not the same person
    possible_supervisors = [eid for eid in all_employees if eid != emp_id]
    supervisor_id = random.choice(possible_supervisors) if possible_supervisors else None

    # Update the employee row
    cursor.execute(
        "UPDATE Employee SET SupervisorID = %s WHERE PersonID = %s",
        (supervisor_id, emp_id)
    )

conn.commit()
print("✅ SupervisorID values updated successfully.")

cursor.close()
conn.close()

-- UPDATE Employee SET SupervisorID
✅ SupervisorID values updated successfully.


In [42]:
import pymysql
import random

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- UPDATE Customer Table")

# Step 1: Fetch all PersonIDs from Employee and Customer
cursor.execute("SELECT PersonID FROM Person")
all_persons = [row[0] for row in cursor.fetchall()]

# Step 2: Fetch all Employee PersonIDs
cursor.execute("SELECT PersonID FROM Employee")
all_employees = [row[0] for row in cursor.fetchall()]

for person_id in all_persons:
    # Choose a random preferred sales rep from Employee
    preferred_sales_rep = random.choice(all_employees) if all_employees else None

    # Insert into Customer table
    cursor.execute(
        "INSERT INTO Customer (PersonID, PreferredSalesRepID) VALUES (%s, %s)",
        (person_id, preferred_sales_rep)
    )

conn.commit()
print("✅ Customer values populated successfully.")

cursor.close()
conn.close()


-- UPDATE Customer Table
✅ Customer values populated successfully.


In [43]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- INSERT INTO PotentialEmployee Table")

# Step 1: Fetch all PersonIDs from Person
cursor.execute("SELECT PersonID FROM Person")
all_persons = [row[0] for row in cursor.fetchall()]

for person_id in all_persons:
    # Insert into PotentialEmployee table
    cursor.execute(
        "INSERT INTO PotentialEmployee (PersonID) VALUES (%s)",
        (person_id,)
    )

conn.commit()
print("✅ PotentialEmployee values populated successfully.")

cursor.close()
conn.close()


-- INSERT INTO PotentialEmployee Table
✅ PotentialEmployee values populated successfully.


In [44]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- INSERT INTO EmployeeDepartmentHistory Table")

# Step 1: Fetch all Employee PersonIDs and DepartmentIDs
cursor.execute("SELECT PersonID FROM Employee")
all_employees = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT DepartmentID FROM Department")
all_departments = [row[0] for row in cursor.fetchall()]

for emp_id in all_employees:
    # Choose a random department
    department_id = random.choice(all_departments) if all_departments else None
    
    # Generate random start time and end time for the department history
    start_time = datetime.now() - timedelta(days=random.randint(30, 365))
    end_time = start_time + timedelta(days=random.randint(30, 365))

    # Insert into EmployeeDepartmentHistory table
    cursor.execute(
        "INSERT INTO EmployeeDepartmentHistory (PersonID, DepartmentID, StartTime, EndTime) VALUES (%s, %s, %s, %s)",
        (emp_id, department_id, start_time.date(), end_time.date())
    )

conn.commit()
print("✅ EmployeeDepartmentHistory values populated successfully.")

cursor.close()
conn.close()


-- INSERT INTO EmployeeDepartmentHistory Table
✅ EmployeeDepartmentHistory values populated successfully.


In [45]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- INSERT INTO Job Table")

# Step 1: Fetch all DepartmentIDs
cursor.execute("SELECT DepartmentID FROM Department")
all_departments = [row[0] for row in cursor.fetchall()]

for dept_id in all_departments:
    # Insert a job for each department
    cursor.execute(
        "INSERT INTO Job (JobID, JobDescription, PostedDate, DepartmentID) VALUES (%s, %s, %s, %s)",
        (random.randint(1, 1000), 'Job description example', datetime.now().date(), dept_id)
    )

conn.commit()
print("✅ Job values populated successfully.")

cursor.close()
conn.close()


-- INSERT INTO Job Table
✅ Job values populated successfully.


In [4]:
import pymysql
import random
from datetime import datetime, timedelta

# Connect to your database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- Updating Job PostedDate values")

# Define date range: Jan 1, 2011 to today
start_date = datetime(2011, 1, 1)
end_date = datetime.now()

# Fetch all JobIDs
cursor.execute("SELECT JobID FROM Job")
job_ids = [row[0] for row in cursor.fetchall()]

# Update each JobID with a random PostedDate in the range
for job_id in job_ids:
    delta_days = (end_date - start_date).days
    random_days = random.randint(0, delta_days)
    random_posted_date = start_date + timedelta(days=random_days)

    cursor.execute(
        "UPDATE Job SET PostedDate = %s WHERE JobID = %s",
        (random_posted_date.date(), job_id)
    )

conn.commit()
print("✅ All Job PostedDate values updated.")

cursor.close()
conn.close()


-- Updating Job PostedDate values
✅ All Job PostedDate values updated.


In [5]:
import pymysql
import random
from datetime import datetime, timedelta

# Connect to database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- Inserting 5 new Job records and updating PostedDate for all")

# Step 1: Fetch all DepartmentIDs
cursor.execute("SELECT DepartmentID FROM Department")
all_departments = [row[0] for row in cursor.fetchall()]

# Step 2: Insert 5 more jobs
new_jobs = []
for _ in range(5):
    dept_id = random.choice(all_departments)
    job_id = random.randint(1001, 9999)  # Avoid duplicates with original 1–1000 range
    cursor.execute(
        "INSERT INTO Job (JobID, JobDescription, PostedDate, DepartmentID) VALUES (%s, %s, %s, %s)",
        (job_id, 'Extra job description', datetime.now().date(), dept_id)
    )
    new_jobs.append(job_id)

# Step 3: Fetch ALL JobIDs now (should be 10 total after insertion)
cursor.execute("SELECT JobID FROM Job")
job_ids = [row[0] for row in cursor.fetchall()]

# Step 4: Select 2 JobIDs to set to January 2011
jan_2011_jobs = random.sample(job_ids, 2)
for job_id in jan_2011_jobs:
    random_day = random.randint(1, 31)
    jan_date = datetime(2011, 1, random_day).date()
    cursor.execute("UPDATE Job SET PostedDate = %s WHERE JobID = %s", (jan_date, job_id))

# Step 5: Update remaining jobs to random dates from 2010 to now
start_date = datetime(2010, 1, 1)
end_date = datetime.now()
for job_id in job_ids:
    if job_id in jan_2011_jobs:
        continue
    delta_days = (end_date - start_date).days
    random_days = random.randint(0, delta_days)
    random_posted_date = start_date + timedelta(days=random_days)
    cursor.execute("UPDATE Job SET PostedDate = %s WHERE JobID = %s", (random_posted_date.date(), job_id))

# Commit and close
conn.commit()
print("✅ 5 Jobs added, and 10 total jobs updated with proper PostedDates.")

cursor.close()
conn.close()


-- Inserting 5 new Job records and updating PostedDate for all
✅ 5 Jobs added, and 10 total jobs updated with proper PostedDates.


In [46]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- INSERT INTO Application Table")

# Step 1: Fetch all PersonIDs from Person
cursor.execute("SELECT PersonID FROM Person")
all_persons = [row[0] for row in cursor.fetchall()]

# Step 2: Fetch all JobIDs
cursor.execute("SELECT JobID FROM Job")
all_jobs = [row[0] for row in cursor.fetchall()]

for person_id in all_persons:
    # Choose a random job for application
    job_id = random.choice(all_jobs) if all_jobs else None
    
    # Insert into Application table
    cursor.execute(
        "INSERT INTO Application (ApplicantID, JobID) VALUES (%s, %s)",
        (person_id, job_id)
    )

conn.commit()
print("✅ Application values populated successfully.")

cursor.close()
conn.close()


-- INSERT INTO Application Table
✅ Application values populated successfully.


In [53]:
from datetime import datetime, timedelta
import pymysql
import random

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

# Fetch all JobIDs from the Job table
cursor.execute("SELECT JobID FROM Job")
valid_job_ids = [row[0] for row in cursor.fetchall()]

# Fetch existing InterviewIDs
cursor.execute("SELECT InterviewID FROM Interview")
existing_ids = [row[0] for row in cursor.fetchall()]

# Generate a unique InterviewID
def generate_unique_interview_id():
    new_id = random.randint(1000, 9999)
    while new_id in existing_ids:
        new_id = random.randint(1000, 9999)
    existing_ids.append(new_id)  # Add to avoid future duplicates
    return new_id

# Generate a random interview time within a 30-day window
def generate_random_interview_time():
    start_date = datetime(2025, 5, 1, 9, 0, 0)  # May 1, 2025, 9 AM
    end_date = start_date + timedelta(days=30)
    random_time = start_date + timedelta(
        days=random.randint(0, 29),
        hours=random.randint(0, 8),    # Interview hours between 9am–5pm
        minutes=random.choice([0, 15, 30, 45])  # Round to quarter-hour
    )
    return random_time.strftime('%Y-%m-%d %H:%M:%S')

# Insert Interview records
for _ in range(10):
    interview_id = generate_unique_interview_id()
    interview_time = generate_random_interview_time()
    job_id = random.choice(valid_job_ids)
    interviewee_id = random.choice([1, 2, 3, 4])  # Replace with valid IDs

    cursor.execute(
        "INSERT INTO Interview (InterviewID, InterviewTime, JobID, IntervieweeID) VALUES (%s, %s, %s, %s)",
        (interview_id, interview_time, job_id, interviewee_id)
    )

conn.commit()
print("✅ Interview values with varied times inserted.")
cursor.close()
conn.close()


✅ Interview values with varied times inserted.


In [54]:

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- INSERT INTO InterviewResult Table")

# Step 1: Fetch all InterviewIDs and Employee PersonIDs
cursor.execute("SELECT InterviewID FROM Interview")
all_interviews = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT PersonID FROM Employee")
all_employees = [row[0] for row in cursor.fetchall()]

for interview_id in all_interviews:
    for interviewer_id in all_employees:
        # Generate random grade between 0 and 100
        grade = random.randint(0, 100)

        # Insert into InterviewResult table
        cursor.execute(
            "INSERT INTO InterviewResult (InterviewID, InterviewerID, Grade) VALUES (%s, %s, %s)",
            (interview_id, interviewer_id, grade)
        )

conn.commit()
print("✅ InterviewResult values populated successfully.")

cursor.close()
conn.close()


-- INSERT INTO InterviewResult Table
✅ InterviewResult values populated successfully.


In [58]:
import pymysql
import random

conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

product_types = ['Electronics', 'Furniture', 'Clothing', 'Appliance', 'Toy']
sizes = ['Small', 'Medium', 'Large']
styles = ['Modern', 'Classic', 'Sporty', 'Vintage']

print("➡️ Inserting into Product table...")
for product_id in range(1, 21):  # Insert 20 products
    product_type = random.choice(product_types)
    size = random.choice(sizes)
    price = round(random.uniform(10.00, 1000.00), 2)
    weight = round(random.uniform(1.0, 50.0), 2)
    style = random.choice(styles)

    cursor.execute(
        "INSERT INTO Product (ProductID, ProductType, Size, ListPrice, Weight, Style) VALUES (%s, %s, %s, %s, %s, %s)",
        (product_id, product_type, size, price, weight, style)
    )

conn.commit()
print("✅ Product table populated.")


➡️ Inserting into Product table...
✅ Product table populated.


In [59]:
site_names = ['Amazon Hub', 'BestBuy Outlet', 'eBay Center', 'Walmart Station', 'Target Spot']
locations = ['New York', 'Los Angeles', 'Dallas', 'Chicago', 'Miami']

print("➡️ Inserting into MarketingSite table...")
for site_id in range(1, 6):  # 5 sites
    site_name = random.choice(site_names)
    site_location = random.choice(locations)

    cursor.execute(
        "INSERT INTO MarketingSite (SiteID, SiteName, SiteLocation) VALUES (%s, %s, %s)",
        (site_id, site_name, site_location)
    )

conn.commit()
print("✅ MarketingSite table populated.")

cursor.close()
conn.close()


➡️ Inserting into MarketingSite table...
✅ MarketingSite table populated.


In [61]:
import pymysql
import random
from datetime import datetime, timedelta

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

# Fetch foreign key values
def fetch_ids(query):
    cursor.execute(query)
    return [row[0] for row in cursor.fetchall()]

salesmen = fetch_ids("SELECT PersonID FROM Employee")
customers = fetch_ids("SELECT PersonID FROM Customer")
products = fetch_ids("SELECT ProductID FROM Product")
sites = fetch_ids("SELECT SiteID FROM MarketingSite")

# Validate
if not all([salesmen, customers, products, sites]):
    print("❌ One or more required tables are empty:")
    if not salesmen: print("- Employee (SalesmanID)")
    if not customers: print("- Customer (CustomerID)")
    if not products: print("- Product (ProductID)")
    if not sites: print("- MarketingSite (SiteID)")
else:
    for sale_id in range(1, 21):  # Create 20 sale records
        salesman_id = random.choice(salesmen)
        customer_id = random.choice(customers)
        product_id = random.choice(products)
        site_id = random.choice(sites)
        sale_time = datetime.now() - timedelta(days=random.randint(0, 30))

        cursor.execute(
            "INSERT INTO Sale (SaleID, SalesmanID, CustomerID, ProductID, SaleTime, SiteID) VALUES (%s, %s, %s, %s, %s, %s)",
            (sale_id, salesman_id, customer_id, product_id, sale_time.strftime('%Y-%m-%d %H:%M:%S'), site_id)
        )

    conn.commit()
    print("✅ Sale records inserted successfully.")

cursor.close()
conn.close()


✅ Sale records inserted successfully.


In [62]:

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()
print("-- INSERT INTO Salary Table")

# Step 1: Fetch all Employee PersonIDs
cursor.execute("SELECT PersonID FROM Employee")
all_employees = [row[0] for row in cursor.fetchall()]

for emp_id in all_employees:
    # Generate random transaction number and amount
    transaction_number = random.randint(1, 1000)
    amount = round(random.uniform(3000, 10000), 2)

    # Insert into Salary table
    cursor.execute(
        "INSERT INTO Salary (PersonID, TransactionNumber, PayDate, Amount) VALUES (%s, %s, %s, %s)",
        (emp_id, transaction_number, datetime.now().date(), amount)
    )

conn.commit()
print("✅ Salary values populated successfully.")

cursor.close()
conn.close()


-- INSERT INTO Salary Table
✅ Salary values populated successfully.


In [66]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()


# Step 1: Fetch InterviewIDs
cursor.execute("SELECT InterviewID FROM Interview")
interview_ids = [row[0] for row in cursor.fetchall()]

# Step 2: Fetch EmployeeIDs
cursor.execute("SELECT PersonID FROM Employee")
employee_ids = [row[0] for row in cursor.fetchall()]

# Step 3: Fetch existing InterviewResult pairs
cursor.execute("SELECT InterviewID, InterviewerID FROM InterviewResult")
existing_pairs = set(cursor.fetchall())

grades = range(40, 100)
new_entries = set()
data_to_insert = []

# Step 4: Create unique, new interview results
for interview_id in interview_ids:
    num_interviewers = random.randint(1, 3)
    interviewers = random.sample(employee_ids, min(num_interviewers, len(employee_ids)))
    for interviewer_id in interviewers:
        pair = (interview_id, interviewer_id)
        if pair not in existing_pairs and pair not in new_entries:
            new_entries.add(pair)
            grade = random.choice(grades)
            data_to_insert.append((interview_id, interviewer_id, grade))

# Step 5: Insert
cursor.executemany(
    "INSERT INTO InterviewResult (InterviewID, InterviewerID, Grade) VALUES (%s, %s, %s)",
    data_to_insert
)
conn.commit()
print("✅ InterviewResult table populated.")


✅ InterviewResult table populated.


In [68]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

vendor_data = []

# Generate 20 fake vendors
for _ in range(20):
    vendor_id = random.randint(1000, 9999)  # Random VendorID
    vname = fake.company()
    address_line1 = fake.street_address()
    address_line2 = fake.secondary_address()
    city = fake.city()
    state = fake.state()
    zip_code = fake.zipcode()
    account_number = fake.iban()  # Fake account number
    credit_rate = random.randint(500, 850)  # Random credit rate between 500 and 850
    web_service_url = fake.url()
    
    vendor_data.append((vendor_id, vname, address_line1, address_line2, city, state, zip_code, account_number, credit_rate, web_service_url))

# Inserting the data into the Vendor table
cursor.executemany(
    "INSERT INTO Vendor (VendorID, Vname, AddressLine1, AddressLine2, City, State, ZipCode, AccountNumber, CreditRate, WebServiceURL) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
    vendor_data
)
conn.commit()
print("✅ Vendor table populated.")




✅ Vendor table populated.


In [ ]:
# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

part_type_data = []

# Generate 10 fake part type entries
for _ in range(10):
    part_type_id = random.randint(1000, 9999)  # Random PartTypeID
    part_name = fake.word()  # Random part name (e.g., "Engine", "Wheel")
    part_type_data.append((part_type_id, part_name))

# Inserting the data into the PartType table
cursor.executemany(
    "INSERT INTO PartType (PartTypeID, PartName) VALUES (%s, %s)",
    part_type_data
)
conn.commit()
print("✅ PartType table populated.")



✅ PartType table populated.


In [83]:

from time import sleep  # Import the sleep function

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

# Fetch all VendorIDs from the Vendor table
cursor.execute("SELECT VendorID FROM Vendor")
vendor_ids = [row[0] for row in cursor.fetchall()]

# Fetch all PartTypeIDs from the PartType table
cursor.execute("SELECT PartTypeID FROM PartType")
part_type_ids = [row[0] for row in cursor.fetchall()]

# Truncate the VendorPartPrice table to clear existing data
cursor.execute("TRUNCATE TABLE VendorPartPrice")
conn.commit()  # Commit the truncation

# Generate random entries for VendorPartPrice
vendor_part_price_data = []

# Generate 10 random entries (adjust the number as needed)
for _ in range(10):
    vendor_id = random.choice(vendor_ids)  # Randomly select a VendorID
    part_type_id = random.choice(part_type_ids)  # Randomly select a PartTypeID
    price = round(random.uniform(50.00, 500.00), 2)  # Random price between 50.00 and 500.00
    vendor_part_price_data.append((vendor_id, part_type_id, price))

# Function to execute the insert with retry logic
def execute_insert_with_retry():
    retries = 3
    batch_size = 100  # Increase batch size for better performance
    total_batches = (len(vendor_part_price_data) + batch_size - 1) // batch_size  # Calculate total batches

    for attempt in range(retries):
        try:
            for i in range(0, len(vendor_part_price_data), batch_size):
                batch = vendor_part_price_data[i:i + batch_size]
                # Use "ON DUPLICATE KEY UPDATE" to avoid the duplicate key error
                cursor.executemany(
                    "INSERT INTO VendorPartPrice (VendorID, PartTypeID, Price) "
                    "VALUES (%s, %s, %s) "
                    "ON DUPLICATE KEY UPDATE Price = VALUES(Price)",  # Update price if duplicate
                    batch
                )
                print(f"Batch {i // batch_size + 1}/{total_batches} inserted.")
            
            conn.commit()  # Commit after all batches are inserted
            print("✅ VendorPartPrice table populated.")
            break  # Exit the retry loop if successful
        except pymysql.MySQLError as e:
            print(f"Error occurred: {e}")
            if attempt < retries - 1:
                print(f"Retrying in 5 seconds... (Attempt {attempt + 1} of {retries})")
                sleep(5)  # Wait for 5 seconds before retrying
            else:
                print("Max retries reached. Operation failed.")

# Call the function
execute_insert_with_retry()

# Close the connection
cursor.close()
conn.close()


Batch 1/1 inserted.
✅ VendorPartPrice table populated.


In [85]:
import pymysql
import random
from time import sleep  # Import the sleep function

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

# Fetch all ProductIDs from the Product table
cursor.execute("SELECT ProductID FROM Product")
product_ids = [row[0] for row in cursor.fetchall()]

# Fetch all PartTypeIDs from the PartType table
cursor.execute("SELECT PartTypeID FROM PartType")
part_type_ids = [row[0] for row in cursor.fetchall()]

# Truncate the ProductParts table to clear existing data
cursor.execute("TRUNCATE TABLE ProductParts")
conn.commit()  # Commit the truncation

# Generate random entries for ProductParts
product_parts_data = []

# Generate 10 random entries (adjust the number as needed)
for _ in range(10):
    product_id = random.choice(product_ids)  # Randomly select a ProductID
    part_type_id = random.choice(part_type_ids)  # Randomly select a PartTypeID
    quantity = random.randint(1, 10)  # Random quantity between 1 and 10
    product_parts_data.append((product_id, part_type_id, quantity))

# Function to execute the insert with retry logic
def execute_insert_with_retry():
    retries = 3
    batch_size = 2  # Insert in smaller batches (e.g., 2 rows at a time)

    for attempt in range(retries):
        try:
            for i in range(0, len(product_parts_data), batch_size):
                batch = product_parts_data[i:i + batch_size]
                # Use "ON DUPLICATE KEY UPDATE" to avoid the duplicate key error
                cursor.executemany(
                    "INSERT INTO ProductParts (ProductID, PartTypeID, Quantity) "
                    "VALUES (%s, %s, %s) "
                    "ON DUPLICATE KEY UPDATE Quantity = VALUES(Quantity)",  # Update quantity if duplicate
                    batch
                )
                conn.commit()  # Commit after each batch
                print(f"Batch {i // batch_size + 1} inserted.")
            print("✅ ProductParts table populated.")
            break  # Exit the retry loop if successful
        except pymysql.MySQLError as e:
            print(f"Error occurred: {e}")
            if attempt < retries - 1:
                print(f"Retrying in 5 seconds... (Attempt {attempt + 1} of {retries})")
                sleep(5)  # Wait for 5 seconds before retrying
            else:
                print("Max retries reached. Operation failed.")

# Call the function
execute_insert_with_retry()

# Close the connection
cursor.close()
conn.close()


Batch 1 inserted.
Batch 2 inserted.
Batch 3 inserted.
Batch 4 inserted.
Batch 5 inserted.
✅ ProductParts table populated.


In [86]:
import pymysql
import random
from time import sleep  # Import the sleep function

# Connect to MySQL database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='9956',
    database='companyxyz'
)
cursor = conn.cursor()

# Fetch all PersonIDs from the Employee table
cursor.execute("SELECT PersonID FROM Employee")
employee_ids = [row[0] for row in cursor.fetchall()]

# Fetch all SiteIDs from the MarketingSite table
cursor.execute("SELECT SiteID FROM MarketingSite")
site_ids = [row[0] for row in cursor.fetchall()]

# Truncate the SiteAssignment table to clear existing data
cursor.execute("TRUNCATE TABLE SiteAssignment")
conn.commit()  # Commit the truncation

# Generate random entries for SiteAssignment
site_assignment_data = []

# Generate 10 random entries (adjust the number as needed)
for _ in range(10):
    person_id = random.choice(employee_ids)  # Randomly select a PersonID
    site_id = random.choice(site_ids)  # Randomly select a SiteID
    site_assignment_data.append((person_id, site_id))

# Function to execute the insert with retry logic
def execute_insert_with_retry():
    retries = 3
    batch_size = 2  # Insert in smaller batches (e.g., 2 rows at a time)

    for attempt in range(retries):
        try:
            for i in range(0, len(site_assignment_data), batch_size):
                batch = site_assignment_data[i:i + batch_size]
                # Use "ON DUPLICATE KEY UPDATE" to avoid the duplicate key error
                cursor.executemany(
                    "INSERT INTO SiteAssignment (PersonID, SiteID) "
                    "VALUES (%s, %s) "
                    "ON DUPLICATE KEY UPDATE SiteID = VALUES(SiteID)",  # Update SiteID if duplicate
                    batch
                )
                conn.commit()  # Commit after each batch
                print(f"Batch {i // batch_size + 1} inserted.")
            print("✅ SiteAssignment table populated.")
            break  # Exit the retry loop if successful
        except pymysql.MySQLError as e:
            print(f"Error occurred: {e}")
            if attempt < retries - 1:
                print(f"Retrying in 5 seconds... (Attempt {attempt + 1} of {retries})")
                sleep(5)  # Wait for 5 seconds before retrying
            else:
                print("Max retries reached. Operation failed.")

# Call the function
execute_insert_with_retry()

# Close the connection
cursor.close()
conn.close()


Batch 1 inserted.
Batch 2 inserted.
Batch 3 inserted.
Batch 4 inserted.
Batch 5 inserted.
✅ SiteAssignment table populated.


In [8]:
import pymysql
from faker import Faker

faker = Faker()
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='9956',
    database='companyxyz',
    autocommit=True  # Helps avoid long open transactions
)

try:
    with connection.cursor() as cursor:
        cursor.execute("SELECT PersonID FROM Person WHERE Email IS NULL")
        person_ids = cursor.fetchall()

        for (person_id,) in person_ids:
            email = faker.unique.email()
            try:
                cursor.execute(
                    "UPDATE Person SET Email = %s WHERE PersonID = %s",
                    (email, person_id)
                )
            except pymysql.err.OperationalError as e:
                print(f"Skipping PersonID {person_id}: {e}")

finally:
    connection.close()
